<a href="https://colab.research.google.com/github/veren4/SMILES_featurization/blob/master/0_Metadata_Preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#First Things First

In this notebook, I determine the max sequence length and the vocabulary.

In [ ]:
import pandas as pd
!pip install -q SmilesPE
from SmilesPE.pretokenizer import atomwise_tokenizer

import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
infile = '/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/CID-SMILES-first100'
#infile = '/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/worded_smiles.csv'
with open(infile, "r") as file1:
  dataset = pd.read_csv(file1,
                        header=None,
                        names=['raw_SMILES'])
file1.close()

In [ ]:
dataset

,raw_SMILES,just_SMILES,tokenized_SMILES
0,1\tCC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,CC(=O)OC(CC(=O)[O-])C[N+](C)(C)C,"[C, C, (, =, O, ), O, C, (, C, C, (, =, O, ), ..."
1,2\tCC(=O)OC(CC(=O)O)C[N+](C)(C)C,CC(=O)OC(CC(=O)O)C[N+](C)(C)C,"[C, C, (, =, O, ), O, C, (, C, C, (, =, O, ), ..."
2,3\tC1=CC(C(C(=C1)C(=O)O)O)O,C1=CC(C(C(=C1)C(=O)O)O)O,"[C, 1, =, C, C, (, C, (, C, (, =, C, 1, ), C, ..."
3,4\tCC(CN)O,CC(CN)O,"[C, C, (, C, N, ), O]"
4,5\tC(C(=O)COP(=O)(O)O)N,C(C(=O)COP(=O)(O)O)N,"[C, (, C, (, =, O, ), C, O, P, (, =, O, ), (, ..."
...,...,...,...
95,99\tCC(=CC(=O)SCCNC(=O)CCNC(=O)C(C(C)(C)COP(=O...,CC(=CC(=O)SCCNC(=O)CCNC(=O)C(C(C)(C)COP(=O)(O)...,"[C, C, (, =, C, C, (, =, O, ), S, C, C, N, C, ..."
96,100\tCC(=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=C...,CC(=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=CCCC(=...,"[C, C, (, =, C, C, C, C, (, =, C, C, C, C, (, ..."
97,101\tC1=CC(=CC(=C1)O)C=O,C1=CC(=CC(=C1)O)C=O,"[C, 1, =, C, C, (, =, C, C, (, =, C, 1, ), O, ..."
98,102\tC1=CC(=CC(=C1)O)CO,C1=CC(=CC(=C1)O)CO,"[C, 1, =, C, C, (, =, C, C, (, =, C, 1, ), O, ..."


In [ ]:
dataset['just_SMILES'] = ''

for row in range(dataset.shape[0]):

  dataset.loc[row, 'just_SMILES'] = re.match(pattern = "\d+\\t(.+)",
                                             string = dataset.loc[row, 'raw_SMILES']).group(1)


In [ ]:
dataset['tokenized_SMILES'] = ''

vocabulary = set()

for row in range(dataset.shape[0]):

  current_tokens = atomwise_tokenizer(dataset.loc[row, 'just_SMILES'])
  dataset.loc[row, 'tokenized_SMILES'] = current_tokens

  vocabulary.update(current_tokens)

In [ ]:
print(vocabulary)

{'(', ')', 'O', 'S', '1', '#', '6', '=', 'C', 'N', '7', '[N+]', '2', '4', 'P', '[O-]', '3', '5', 'Cl'}


0 = SOL\
1 = EOL\
2 = UNK

In [ ]:
from collections import Counter

data = {'Sentences':  ['This is the first sencence.', 'This is another cell', 'Ah, might as well.']}
df = pd.DataFrame (data, columns = ['Sentences'])

text = df['Sentences'].str.cat(sep=' ')
words = text.split(' ')

In [ ]:
words

['This',
 'is',
 'the',
 'first',
 'sencence.',
 'This',
 'is',
 'another',
 'cell',
 'Ah,',
 'might',
 'as',
 'well.']

In [ ]:
type(a)

list

In [ ]:
word_counts = Counter(words)
uniq_words = sorted(word_counts, key=word_counts.get, reverse=True)

In [ ]:
uniq_words

['This',
 'is',
 'the',
 'first',
 'sencence.',
 'another',
 'cell',
 'Ah,',
 'might',
 'as',
 'well.']

In [ ]:
type(uniq_words)

list

In [ ]:
vocabulary_list = list(vocabulary)

In [ ]:
vocabulary_list

['1', 'N', ')', 'C', 'S', '=', '4', 'O', '(', '2', '3', 'P']

In [ ]:
unique_words = ['UNK', 'SOL', 'EOL', 'PAD']
unique_words.extend(vocabulary_list)

In [ ]:
print(unique_words)

['UNK', 'SOL', 'EOL', 'PAD', '1', 'N', ')', 'C', 'S', '=', '4', 'O', '(', '2', '3', 'P']


In [ ]:
len(unique_words)

16

In [ ]:
word_to_index = {word: index for index, word in enumerate(unique_words)}

In [ ]:
word_to_index

{'(': 12,
 ')': 6,
 '1': 4,
 '2': 13,
 '3': 14,
 '4': 10,
 '=': 9,
 'C': 7,
 'EOL': 2,
 'N': 5,
 'O': 11,
 'P': 15,
 'PAD': 3,
 'S': 8,
 'SOL': 1,
 'UNK': 0}

In [ ]:
test = dataset.tokenized_SMILES[1]

In [ ]:
words_indexes = [word_to_index[w] for w in test]

In [ ]:
words_indexes

[7, 7, 7, 12, 7, 6, 7, 12, 9, 11, 6, 7, 12, 9, 11, 6, 11]

In [ ]:
'bla' in unique_words

False

In [ ]:
word_to_index['UNK']

0

#Padding

In [ ]:
max_sequence_length = 150

In [ ]:
        infile = '/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/worded_smiles.csv'
        with open(infile, "r") as file1:
            train_df = pd.read_csv(file1)
        file1.close()

        # Tokenize
        train_df['tokenized_SMILES'] = ''
        for row in range(train_df.shape[0]):
          train_df.loc[row, 'tokenized_SMILES'] = atomwise_tokenizer(train_df.loc[row, 'SMILES'])
        
        # Padding + SOL + EOL
        for row in range(train_df.shape[0]):              # ATTENTION: Are the column indexes correct?!
          actual_length = len(train_df.loc[row, 'tokenized_SMILES'])
          length_before_delimiters = max_sequence_length - 2

          if actual_length > length_before_delimiters:
            train_df.loc[row, 'tokenized_SMILES'] = train_df.loc[row, 'tokenized_SMILES'][:length_before_delimiters]
            train_df.loc[row, 'tokenized_SMILES'].append('EOL')
          elif actual_length < length_before_delimiters:
            temp = ['UNK']*length_before_delimiters
            shortie = train_df.loc[row, 'tokenized_SMILES']
            shortie.append('EOL')
            temp[:actual_length] = shortie
            train_df.loc[row, 'tokenized_SMILES'] = temp

          train_df.loc[row, 'tokenized_SMILES'].insert(0, 'SOL')

In [ ]:
#train_df

In [ ]:
total_token_list = []

#for row in range(train_df.shape[0]):
for row in range(2):
  total_token_list.extend(train_df.loc[row, 'tokenized_SMILES'])

#Filestream

In [ ]:
import torch
import pandas as pd
from collections import Counter
from torch import nn, optim
import argparse
import numpy as np
from torch.utils.data import DataLoader

!pip install -q SmilesPE
from SmilesPE.pretokenizer import atomwise_tokenizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import platform
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print('Python: ', platform.python_version())
print('PyTorch: ', torch.__version__)
if(device.type == 'cuda'):
  print('Using GPU (cuda)')
else:
  print('Using CPU!')

Python:  3.6.9
PyTorch:  1.7.0+cu101
Using CPU!


In [ ]:
data_path = '/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/worded_smiles.csv'

data_stream = open(data_path, 'r')

offset = 9

In [ ]:
#offset = self.meta['offset_dict'][line]
offset =
data_stream.seek(offset)
line = data_stream.readline()
instance = line
offset = offset + len(line)

instance

'"C C C ( C ) C ( = O ) C ( = O ) O"\n'

In [ ]:
len(instance)

68

In [ ]:
68+9

77

In [ ]:
infile = '/content/drive/My Drive/Rostlab internship/8_KDNuggets_LSTM_Approach/data/worded_smiles.csv'
with open(infile, "r") as file1:
  train_df = pd.read_csv(file1)
file1.close()

In [ ]:
train_df.head(3)

,SMILES
0,C ( C ( C ( C O P ( = O ) ( O ) O ) O ) O ) C ...
1,C C C ( C ) C ( = O ) C ( = O ) O
2,C C ( C ) ( C O P ( = O ) ( O ) O P ( = O ) ( ...


In [ ]:
instance

'"C C C ( C ) C ( = O ) C ( = O ) O"\n'

In [ ]:
b = atomwise_tokenizer(instance)

In [ ]:
type(b)

list

#Random testing

In [ ]:
import torch

In [ ]:
a = [4,5,7,3,2,0,7,4]
sequence_length = 15
a += [-1] * (sequence_length - len(a))
a

[4, 5, 7, 3, 2, 0, 7, 4, -1, -1, -1, -1, -1, -1, -1]

In [ ]:
b = torch.tensor(a)
b

tensor([ 4,  5,  7,  3,  2,  0,  7,  4, -1, -1, -1, -1, -1, -1, -1])

find index of first -1
index -1
slice until there


do again, until there are no more -1s in the tensor.

In [ ]:
a.index(-1)

8

In [ ]:
b

tensor([ 4,  5,  7,  3,  2,  0,  7,  4, -1, -1, -1, -1, -1, -1, -1])

In [ ]:
c = b.numpy()
c

array([ 4,  5,  7,  3,  2,  0,  7,  4, -1, -1, -1, -1, -1, -1, -1])

In [ ]:
import numpy as np

In [ ]:
15 in c

False

In [ ]:
result = np.where(c == -1)
result

(array([ 8,  9, 10, 11, 12, 13, 14]),)

In [ ]:
result[0][0]

8

In [ ]:
first_part = c[:(result[0][0]-1)]

In [ ]:
where(b == -1)

NameError: ignored

In [ ]:
first_part

array([4, 5, 7, 3, 2, 0, 7])

Slicing: https://railsware.com/blog/python-for-machine-learning-indexing-and-slicing-for-lists-tuples-strings-and-other-sequential-types/

In [ ]:
c

array([ 4,  5,  7,  3,  2,  0,  7,  4, -1, -1, -1, -1, -1, -1, -1])

In [ ]:
numpy_version = b.numpy()

result = np.where(numpy_version == -1)
while -1 in numpy_version:
  numpy_version = numpy_version[:(result[0][0]-1)]

d = torch.tensor(numpy_version)

d

tensor([4, 5, 7, 3, 2, 0, 7])

In [ ]:
numpy_version

array([4, 5, 7, 3, 2, 0, 7])

In [ ]:
d

tensor([4, 5, 7, 3, 2, 0, 7])

In [ ]:
raise ValueError('Oh no!')

ValueError: ignored

In [ ]:
instance = [3,1,6,7,8]
sequence_length = 10

In [ ]:
instance += [-1] * (sequence_length - len(instance))
instance

[3, 1, 6, 7, 8, -1, -1, -1, -1, -1]